In [89]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import random
from sklearn.ensemble import AdaBoostClassifier

In [61]:
bigdata = pd.read_csv('bigdata(tf-idf).csv')

In [62]:
bigdata.head()

,Unnamed: 0,index,genre,bookId,entity.n.01,cheerfulness.n.01,eidos.n.01,ethos.n.01,inheritance.n.04,personality.n.01,...,deodorant.n.01,hairdressing.n.01,lotion.n.01,perfume.n.02,weaponry.n.01,ammunition.n.01,armament.n.01,gunnery.n.01,hardware.n.01,totalWords
0,0,0,1,14976,0.0,0.00000,0.0,0.0,0.000000,0.569121,...,0.0,0.0,0.0,0.0,0.0,1.529576,0.0,0.0,0.0,24106
1,1,1,1,2750,0.0,0.00000,0.0,0.0,0.180254,0.102075,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,44801
2,2,2,1,2754,0.0,0.00000,0.0,0.0,0.198778,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,20313
3,3,3,1,2747,0.0,0.63268,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.403016,0.0,0.0,0.0,18298
4,4,4,1,2749,0.0,0.00000,0.0,0.0,0.000000,0.156532,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,29215


In [63]:
msk = np.random.rand(len(bigdata)) < 0.8
train = bigdata[msk]
test = bigdata[~msk]

In [64]:
train.shape

(3741, 772)

In [65]:
test.shape

(939, 772)

In [66]:
temp = bigdata

In [67]:
genre = temp['genre']

In [68]:
del temp['genre']
del temp['Unnamed: 0']

In [69]:
del temp['index']

In [70]:
del temp['bookId']

In [71]:
del temp['totalWords']

In [72]:
Words = temp.values
Words

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.26971441,  0.        ,  0.        , ...,  0.13024733,
         0.        ,  0.        ],
       [ 0.38998832,  0.1584884 ,  0.        , ...,  0.0627762 ,
         0.        ,  0.        ],
       [ 0.13852702,  0.05629637,  0.        , ...,  0.15609031,
         0.        ,  0.        ]])

In [73]:
pca = PCA(n_components=80)
ModelPca = pca.fit(Words)

In [74]:
pca

PCA(copy=True, n_components=80, whiten=False)

In [75]:
print pca.explained_variance_ratio_.sum()

0.905455552103


In [76]:
df=pd.DataFrame()
for i in range(pca.explained_variance_ratio_.shape[0]):
    df["pc%i" % (i+1)] = Words[:,i]
df.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc71,pc72,pc73,pc74,pc75,pc76,pc77,pc78,pc79,pc80
0,0.0,0.00000,0.0,0.0,0.000000,0.569121,5.979216,2.928555,1.052789,0.100223,...,0.420190,0.000000,0.0,0.000000,0.0,0.0,0.286031,0.0,0.125341,0.0
1,0.0,0.00000,0.0,0.0,0.180254,0.102075,8.470673,7.049464,0.740772,0.107853,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.205206,0.0,0.337210,0.0
2,0.0,0.00000,0.0,0.0,0.198778,0.000000,6.466968,4.755807,0.192211,0.000000,...,0.166217,0.434962,0.0,3.293205,0.0,0.0,0.113147,0.0,0.743728,0.0
3,0.0,0.63268,0.0,0.0,0.000000,0.000000,8.575059,4.365759,1.493646,0.396103,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.330251,0.0
4,0.0,0.00000,0.0,0.0,0.000000,0.156532,7.119365,7.694393,0.735038,0.165392,...,0.231139,0.000000,0.0,0.000000,0.0,0.0,0.236011,0.0,0.620531,0.0


In [77]:
bigdata

,entity.n.01,cheerfulness.n.01,eidos.n.01,ethos.n.01,inheritance.n.04,personality.n.01,property.n.02,quality.n.01,shape.n.02,space.n.01,...,cream.n.03,deodorant.n.01,hairdressing.n.01,lotion.n.01,perfume.n.02,weaponry.n.01,ammunition.n.01,armament.n.01,gunnery.n.01,hardware.n.01
0,0.000000,0.000000,0.0,0.0,0.000000,0.569121,5.979216,2.928555,1.052789,0.100223,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.529576,0.000000,0.0,0.000000
1,0.000000,0.000000,0.0,0.0,0.180254,0.102075,8.470673,7.049464,0.740772,0.107853,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.0,0.0,0.198778,0.000000,6.466968,4.755807,0.192211,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.632680,0.0,0.0,0.000000,0.000000,8.575059,4.365759,1.493646,0.396103,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.403016,0.000000,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,0.000000,0.156532,7.119365,7.694393,0.735038,0.165392,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,0.000000,0.000000,0.0,0.0,0.323626,0.122177,10.699340,10.497490,0.286857,1.065014,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,0.000000,0.000000,0.0,0.0,0.000000,0.000000,7.469392,3.457126,0.484376,0.099908,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.327368,0.0,0.000000
7,0.000000,0.000000,0.0,0.0,0.000000,0.109803,7.622499,7.003263,0.374989,0.116018,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,0.000000,0.000000,0.0,0.0,0.187472,0.353877,14.090169,8.596852,0.574051,0.037391,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,0.000000,0.000000,0.0,0.0,0.000000,0.000000,4.496041,10.529572,0.721610,0.893039,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.564994,0.0,2.497082


In [85]:
X_train, X_test, y_train, y_test = train_test_split(df, genre, test_size=0.20, random_state=42)

In [90]:
random.seed(100)
rf =AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=2600)
rf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=2600, random_state=None)

In [91]:
pre=rf.predict(X_test)
accuracy_score(y_test, pre)

0.55448717948717952

In [93]:
y_train

1479    3
3109    3
4100    4
4248    5
831     2
1253    2
1421    3
251     2
1554    3
3129    3
1740    3
680     2
3218    3
3868    4
1158    2
1018    2
3684    4
3557    4
332     2
3289    3
751     2
720     2
4459    7
1769    3
2652    3
3693    4
1773    3
1322    2
1815    3
517     2
       ..
1267    2
1899    3
3005    3
2734    3
189     6
1806    3
975     2
2747    3
2047    3
2558    3
1082    2
474     2
2904    3
4117    4
3385    4
4555    7
1184    2
2433    3
2391    3
769     2
1685    3
130     6
2919    3
3171    3
3444    4
4426    7
466     2
3092    3
3772    4
860     2
Name: genre, dtype: int64